In [1]:
import pandas as pd             # data package
import matplotlib.pyplot as plt # graphics 
import datetime as dt
import numpy as np

import requests, io             # internet and input tools  
import zipfile as zf            # zip file tools 
import os  

#import weightedcalcs as wc
#import numpy as np

import pyarrow as pa
import pyarrow.parquet as pq

In [2]:
date = "2025-11"

my_key = "&key=34e40301bda77077e24c859c6c6c0b721ad73fc7"
# This is my key. I'm nice and I have it posted. If you will be doing more with this
# please get your own key!

In [15]:
end_use = "naics?get=CON_VAL_MO,CTY_CODE,CTY_NAME,SUMMARY_LVL"

url = "https://api.census.gov/data/timeseries/intltrade/imports/" + end_use 
url = url + my_key + "&time==from+2013-01"

r = requests.get(url) 
    
print(r)
    
df = pd.DataFrame(r.json()[1:]) # This then converts it to a dataframe
    # Note that the first entry is the labels

df.columns = r.json()[0]

df["total_imports"] = df["CON_VAL_MO"].astype(float)

df = df[df.SUMMARY_LVL == "DET"]

grp = df.groupby(["CTY_NAME"])

top_products = grp.agg({"total_imports":"sum","CTY_CODE":"first"})

country_list = list(top_products.sort_values(by = "total_imports", ascending = False).CTY_CODE)[0:31]


['TOTAL FOR ALL COUNTRIES','NAFTA','EUROPEAN UNION']

<Response [200]>


['TOTAL FOR ALL COUNTRIES', 'NAFTA', 'EUROPEAN UNION']

In [16]:
df.tail()

,CON_VAL_MO,CTY_CODE,CTY_NAME,SUMMARY_LVL,time,total_imports
39049,13896714,7940,ZAMBIA,DET,2025-11,13896714.0
39050,12624834,7950,ESWATINI,DET,2025-11,12624834.0
39051,15520151,7960,ZIMBABWE,DET,2025-11,15520151.0
39052,2368430,7970,MALAWI,DET,2025-11,2368430.0
39053,8270767,7990,LESOTHO,DET,2025-11,8270767.0


In [17]:
df.time.unique()[-1]

'2025-11'

In [18]:
country_list[0] = ""

In [19]:
country_list.extend(["0003", "0020"])

In [ ]:
end_use = "naics?get=CTY_NAME,CON_VAL_MO,CAL_DUT_MO,NAICS,NAICS_SDESC"

surl = "https://api.census.gov/data/timeseries/intltrade/imports/" + end_use 

surl  = surl + my_key + "&time=" + "from+2013-01" + "&COMM_LVL=NA3" 

for xxx in country_list:
    
    out_file = "..\\data"+ "\\imports\\" + xxx + "naics-data-" + date + ".parquet"
    
    if xxx == "":
        out_file = "..\\data"+ "\\imports\\" + "TOTAL" + "naics-data-" + date + ".parquet"
    
    
    if os.path.exists(out_file):
        
        print("Already have downloaded file")
        
        continue
    
    print(xxx)
    
    url = surl + "&CTY_CODE=" + xxx
    
    if xxx == "":
        url = surl
    
    r = requests.get(url) 
    
    print(r)
    
    foo = pd.DataFrame(r.json()[1:]) # This then converts it to a dataframe
    # Note that the first entry is the labels

    foo.columns = r.json()[0]

    pq.write_table(pa.Table.from_pandas(foo), out_file)

In [11]:
foo

,CTY_NAME,CON_VAL_MO,CAL_DUT_MO,I_COMMODITY,I_COMMODITY_SDESC,time,COMM_LVL,CTY_CODE
0,USMCA (NAFTA),182150721,14,01,LIVE ANIMALS,2013-01,HS2,0020
1,USMCA (NAFTA),212532311,156,01,LIVE ANIMALS,2013-02,HS2,0020
2,USMCA (NAFTA),244602053,57,01,LIVE ANIMALS,2013-03,HS2,0020
3,USMCA (NAFTA),250009423,199,01,LIVE ANIMALS,2013-04,HS2,0020
4,USMCA (NAFTA),182211881,50,01,LIVE ANIMALS,2013-05,HS2,0020
...,...,...,...,...,...,...,...,...
15183,USMCA (NAFTA),2566435182,74731772,90,"OPTIC, PHOTO ETC, MEDIC OR SURGICAL INSTRMENTS...",2025-07,HS2,0020
15184,USMCA (NAFTA),2349482121,74050235,90,"OPTIC, PHOTO ETC, MEDIC OR SURGICAL INSTRMENTS...",2025-08,HS2,0020
15185,USMCA (NAFTA),2424794624,75974180,90,"OPTIC, PHOTO ETC, MEDIC OR SURGICAL INSTRMENTS...",2025-09,HS2,0020
15186,USMCA (NAFTA),2462330395,68896539,90,"OPTIC, PHOTO ETC, MEDIC OR SURGICAL INSTRMENTS...",2025-10,HS2,0020


In [10]:
country_list[0] = "TOTAL"

df = pd.DataFrame([])
#this is for all countries...

for xxx in country_list:
    
    file = "..\\data\\imports\\" + xxx + "naics-data-" + date + ".parquet"

    foo = pq.read_table(file).to_pandas()
        
    df = pd.concat([df, foo], ignore_index=True)
            
df.time = pd.to_datetime(df.time, format="%Y-%m")

df["imports"] = df["CON_VAL_MO"].astype(float)

df["duty"] = df["CAL_DUT_MO"].astype(float)

total = df.groupby(["CTY_NAME", "time"]).agg({"imports": "sum", "duty": "sum"})
                   
total.reset_index(inplace = True)
                   
total["I_COMMODITY"] = "00"
total["I_COMMODITY_SDESC"] = "ALL PRODUCTS"
                   
df = pd.concat([df, total], ignore_index=True)

df["itariff"] = 100*df["duty"]/df["imports"]

df.drop(["CON_VAL_MO", "COMM_LVL", "CTY_CODE", "CAL_DUT_MO"], axis = 1, inplace = True)

df["I_COMMODITY_SDESC"] = "HS CODE " + df["I_COMMODITY"] + ", " + df["I_COMMODITY_SDESC"]

df["I_COMMODITY_SDESC"] = df["I_COMMODITY_SDESC"].replace({"HS CODE 00, ALL PRODUCTS": "ALL PRODUCTS"})

df.set_index(["CTY_NAME","I_COMMODITY_SDESC","time"],inplace = True)

out_file = ".\\data"+ "\\top30-NAICS-imports.parquet"

pq.write_table(pa.Table.from_pandas(df), out_file)

FileNotFoundError: [WinError 3] Failed to open local file './data/top30-NAICS-imports.parquet'. Detail: [Windows error 3] The system cannot find the path specified.


In [ ]:
df.index.get_level_values(0).unique()

Index(['TOTAL FOR ALL COUNTRIES', 'CHINA', 'MEXICO', 'CANADA', 'JAPAN',
       'GERMANY', 'KOREA, SOUTH', 'VIETNAM', 'TAIWAN', 'IRELAND', 'INDIA',
       'UNITED KINGDOM', 'ITALY', 'FRANCE', 'SWITZERLAND', 'THAILAND',
       'MALAYSIA', 'BRAZIL', 'SINGAPORE', 'NETHERLANDS', 'INDONESIA', 'ISRAEL',
       'BELGIUM', 'SAUDI ARABIA', 'SPAIN', 'RUSSIA', 'COLOMBIA', 'AUSTRALIA',
       'AUSTRIA', 'SWEDEN', 'CHILE', 'EUROPEAN UNION', 'USMCA (NAFTA)'],
      dtype='object', name='CTY_NAME')

In [ ]:
df.loc["USMCA (NAFTA)","ALL PRODUCTS"]

c:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: PerformanceWarning: indexing past lexsort depth may impact performance.
  """Entry point for launching an IPython kernel.


,I_COMMODITY,imports,duty,itariff
time,,,,
2013-01-01,00,4.902706e+10,4.498723e+07,0.091760
2013-02-01,00,4.736636e+10,3.666211e+07,0.077401
2013-03-01,00,5.124229e+10,3.717834e+07,0.072554
2013-04-01,00,5.216375e+10,3.709481e+07,0.071112
2013-05-01,00,5.301765e+10,4.009879e+07,0.075633
...,...,...,...,...
2025-07-01,00,7.721823e+10,3.100668e+09,4.015461
2025-08-01,00,7.433524e+10,3.242082e+09,4.361433
2025-09-01,00,7.621011e+10,3.336693e+09,4.378281


In [ ]:
df.loc["USMCA (NAFTA)","ALL PRODUCTS"].loc["2024"]

c:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: PerformanceWarning: indexing past lexsort depth may impact performance.
  """Entry point for launching an IPython kernel.


,I_COMMODITY,imports,duty,itariff
time,,,,
2024-01-01,00,7.123938e+10,133676383.0,0.187644
2024-02-01,00,7.323562e+10,131136228.0,0.179061
2024-03-01,00,7.548168e+10,145216344.0,0.192386
2024-04-01,00,7.720258e+10,141640513.0,0.183466
2024-05-01,00,7.931487e+10,147193371.0,0.185581
2024-06-01,00,7.663457e+10,134896847.0,0.176026
2024-07-01,00,7.740682e+10,144363663.0,0.186500
2024-08-01,00,7.626432e+10,142907689.0,0.187385
2024-09-01,00,7.851141e+10,130233104.0,0.165878


In [ ]:
foo = pq.read_table(".\\data\\top30-HS2-imports.parquet").to_pandas()

In [ ]:
foo.tail()

I_COMMODITY       imports          duty  \
CTY_NAME I_COMMODITY_SDESC time                                                 
VIETNAM  ALL PRODUCTS      2025-07-01          00  1.776576e+10  1.640984e+09   
                           2025-08-01          00  1.723779e+10  1.722283e+09   
                           2025-09-01          00  1.858137e+10  2.343462e+09   
                           2025-10-01          00  1.958902e+10  2.412794e+09   
                           2025-11-01          00  1.794097e+10  2.185444e+09   

                                         itariff  
CTY_NAME I_COMMODITY_SDESC time                   
VIETNAM  ALL PRODUCTS      2025-07-01   9.236779  
                           2025-08-01   9.991325  
                           2025-09-01  12.611886  
                           2025-10-01  12.317073  
                           2025-11-01  12.181308

In [ ]:
foo.loc["TOTAL FOR ALL COUNTRIES", "HS CODE 01, LIVE ANIMALS"]

c:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: PerformanceWarning: indexing past lexsort depth may impact performance.
  """Entry point for launching an IPython kernel.


,I_COMMODITY,imports,duty,itariff
time,,,,
2013-01-01,01,221313094.0,417.0,0.000188
2013-02-01,01,246092109.0,823.0,0.000334
2013-03-01,01,261244392.0,75.0,0.000029
2013-04-01,01,271133926.0,1389.0,0.000512
2013-05-01,01,209785289.0,50.0,0.000024
...,...,...,...,...
2025-07-01,01,266619048.0,3777405.0,1.416780
2025-08-01,01,215515069.0,3748886.0,1.739501
2025-09-01,01,273013211.0,4814765.0,1.763565
